In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import xarray as xr
import numpy as np
import pandas as pd
from scipy import stats

import ecephys_analyses as ea 
from ecephys.signal.xarray_utils import filter_dataset_by_hypnogram

In [4]:
xr.set_options(keep_attrs=True)

In [5]:
subjects = ["Segundo", "Valentino", "Doppio", "Alessandro", "Allan"]
experiment = "sleep-homeostasis"
condition = "recovery-sleep"
states = ["N1", "N2"]
duration = '02:00:00'
f_range = (0.5, 4)

In [6]:
def get_subject_data(
    subject, experiment, condition, states, duration, f_range
):
    hyp = ea.load_hypnogram(subject, experiment, condition)
    power = ea.load_power(subject, experiment, condition, "spg.nc").sel(frequency=slice(*f_range)).sum(dim='frequency')
    
    early_hyp = hyp.keep_states(states).keep_first(duration)
    early_pow = filter_dataset_by_hypnogram(power, early_hyp).swap_dims({'time': 'datetime'}).drop_vars(['time', 'timedelta'])

    late_hyp = hyp.keep_states(states).keep_last(duration)
    late_pow = filter_dataset_by_hypnogram(power, late_hyp).swap_dims({'time': 'datetime'}).drop_vars(['time', 'timedelta'])

    diff_means = early_pow.mean() - late_pow.mean()
    diff_means_std = diff_means / late_pow.mean()
    diff_meds = early_pow.median() - late_pow.median()
    diff_meds_std = diff_meds / late_pow.median()

    index = pd.MultiIndex.from_product(
        [[subject], ["cortex", "hippocampus"]], names=["subject", "region"]
    )
    data = np.asarray(
        [
            [
                diff_means.mpta_wm_ref,
                diff_means_std.mpta_wm_ref,
                diff_meds.mpta_wm_ref,
                diff_meds_std.mpta_wm_ref,
            ],
            [
                diff_means.sr_wm_ref,
                diff_means_std.sr_wm_ref,
                diff_meds.sr_wm_ref,
                diff_meds_std.sr_wm_ref,
            ],
        ]
    )
    return pd.DataFrame(
        data,
        columns=["diff_means", "diff_means_std", "diff_meds", "diff_meds_std"],
        index=index,
    )

In [7]:
subj_data = list()
for subject in subjects:
    subj_data.append(get_subject_data(subject, experiment, condition, states, duration, f_range))
df = pd.concat(subj_data)

In [8]:
print(df.to_markdown())

|                               |   diff_means |   diff_means_std |   diff_meds |   diff_meds_std |
|:------------------------------|-------------:|-----------------:|------------:|----------------:|
| ('Segundo', 'cortex')         |     184050   |         0.469843 |    216467   |        0.676873 |
| ('Segundo', 'hippocampus')    |      82041.2 |         0.192321 |     85454   |        0.230777 |
| ('Valentino', 'cortex')       |      58629.3 |         0.589774 |     49505.3 |        0.696541 |
| ('Valentino', 'hippocampus')  |      77991.6 |         0.157365 |     96414.5 |        0.276582 |
| ('Doppio', 'cortex')          |     169282   |         0.454567 |    159759   |        0.504246 |
| ('Doppio', 'hippocampus')     |      61441.2 |         0.197231 |     55584   |        0.234934 |
| ('Alessandro', 'cortex')      |     124200   |         0.504609 |    146674   |        0.683805 |
| ('Alessandro', 'hippocampus') |     141682   |         0.336182 |    171404   |        0.521517 |


In [9]:
cx = df.xs('cortex', level=1, drop_level=False)
hc = df.xs('hippocampus', level=1, drop_level=False)

In [10]:
stats.ttest_1samp(cx, 0)

Ttest_1sampResult(statistic=array([ 4.1371808 , 20.05991872,  3.94446014, 17.3145766 ]), pvalue=array([1.44094971e-02, 3.64479880e-05, 1.68963764e-02, 6.52991239e-05]))

In [11]:
stats.ttest_1samp(hc, 0)

Ttest_1sampResult(statistic=array([5.51054499, 6.71541183, 4.39284184, 5.41839367]), pvalue=array([0.00529142, 0.00255999, 0.01175654, 0.00562301]))